In [1]:
%load_ext autoreload
%autoreload 2

from imports_paper_plots import *
set_context_and_font()

plt.rcParams["axes.facecolor"] = "1."

In [2]:
results_dir_path = Path(os.environ["DATAICNS"]) / "bioRNN/memory-sequence"
figs_path = Path(os.environ["DATAICNS"]) / "bioRNN/memory-sequence/figs"

In [3]:
neuron_density = 1
results_hyperparams_path = results_dir_path/"hyperparams"/f"neuron-density-{neuron_density}"/"hyperparams_all"

results = pd.read_csv(results_hyperparams_path)
results.head(3)

,connectome,rand_partition,neuron_density,W,activation_out,bias,input_scaling,leak_rate,n_transient,pattern_length,spectral_radius,r2
0,macaque,True,1,bio_rank,CPUDispatcher(<function relu at 0x7f36c8eb81f0>),0,0.000001,0.6,100,5,0.91,0.999160
1,macaque,True,1,bio_rank,CPUDispatcher(<function relu at 0x7f36c8eb81f0>),0,0.000001,0.6,100,5,0.93,0.999623
2,macaque,True,1,bio_rank,CPUDispatcher(<function relu at 0x7f36c8eb81f0>),0,0.000001,0.6,100,5,0.95,0.999932


In [4]:
results = results.reindex(
    columns=['connectome', 'rand_partition', 'neuron_density', 
             'W', 'input_scaling', 'leak_rate', 
             'spectral_radius', 'pattern_length', 'r2']
)
results.head(3)

,connectome,rand_partition,neuron_density,W,input_scaling,leak_rate,spectral_radius,pattern_length,r2
0,macaque,True,1,bio_rank,0.000001,0.6,0.91,5,0.999160
1,macaque,True,1,bio_rank,0.000001,0.6,0.93,5,0.999623
2,macaque,True,1,bio_rank,0.000001,0.6,0.95,5,0.999932


In [5]:
(results.connectome.unique(),
results.rand_partition.unique(),
results.input_scaling.unique(),
results.pattern_length.unique(),)

(array(['macaque', 'marmoset', 'human'], dtype=object),
 array([ True, False]),
 array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
 array([ 5, 10, 15, 20, 25, 30]))

## Hyperparameters grid

In [6]:
results.head()

,connectome,rand_partition,neuron_density,W,input_scaling,leak_rate,spectral_radius,pattern_length,r2
0,macaque,True,1,bio_rank,0.000001,0.6,0.91,5,0.999160
1,macaque,True,1,bio_rank,0.000001,0.6,0.93,5,0.999623
2,macaque,True,1,bio_rank,0.000001,0.6,0.95,5,0.999932
3,macaque,True,1,bio_rank,0.000001,0.6,0.97,5,0.999262
4,macaque,True,1,bio_rank,0.000001,0.6,0.99,5,0.999833


In [7]:
results = refactor_names(
    df=results,
    capitalize_cols=["connectome"],
    to_rename=None
)
results.head(3)

,connectome,rand_partition,neuron_density,W,input_scaling,leak_rate,spectral_radius,pattern_length,r2
0,Macaque,True,1,Bio (rank),0.000001,0.6,0.91,5,0.999160
1,Macaque,True,1,Bio (rank),0.000001,0.6,0.93,5,0.999623
2,Macaque,True,1,Bio (rank),0.000001,0.6,0.95,5,0.999932


In [8]:
%%time
connectomes = ["Macaque", "Human", "Marmoset"]
rand_partitions = [True, False,]
pattern_lengths = range(5, 31, 5)
for connectome, rand_partition, pattern_length in product(
    connectomes, rand_partitions, pattern_lengths
):
    data = results[(results.connectome == connectome) &
                   (results.rand_partition == rand_partition) &
                   (results.pattern_length == pattern_length)]

    g = plot.hyperparams_grid(
        data=data,
        x="spectral_radius",
        y="r2",
        palette=my_palette,
        facet_kws={"ylim": (-2.5, 1)}
    )
    
    g.set_axis_labels(y_var=r"$R^{2}$", x_var=r"Spectral radius ($\rho$)",)
    set_facetgrid_titles(
        g,
        col_template=r'Input scaling ($\epsilon$) = {col_name}',
        row_template=r"Leak rate ($\alpha$) = {row_name}", 
        size=23
    )
   
    g.fig.subplots_adjust(wspace=0.1, hspace=.003)
    plt.suptitle(f"{connectome}", x=.45, y=1., fontsize=40)

    g.savefig(
        figs_path
        /f"hyperparams/neuron-density-{neuron_density}"
        /f"{connectome}_rand-part-{rand_partition}_patt-len-{pattern_length}"
    )
    plt.close()

CPU times: user 6min 33s, sys: 6.36 s, total: 6min 39s
Wall time: 6min 33s
